In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
from LEN_Scorer import Scorer

In [2]:
mimicDF = pd.read_csv('../LEN_Test/data/StaticData.csv')
targetDF = mimicDF[['deathperiod']]
mimicDF = mimicDF.drop(columns=['PatientID', 'deathperiod'])

In [3]:
mimicDF.head()

,los,gender,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
0,9,1,74.19,7,7,0.0,1.0,NaN,0.0,3.0,3.0
1,2,0,75.00,4,2,NaN,0.0,0.0,1.0,NaN,1.0
2,16,0,51.92,4,2,NaN,0.0,NaN,0.0,NaN,2.0
3,12,0,51.73,-13,4,2.0,2.0,NaN,0.0,NaN,0.0
4,7,0,62.07,0,3,0.0,2.0,NaN,1.0,NaN,0.0


In [4]:
mimicDF.describe()

,los,gender,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
count,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,3311.000000,5227.000000,2254.000000,5261.000000,1294.000000,5212.000000
mean,14.668757,0.382554,75.049002,9.801026,4.794185,1.987315,0.674957,0.696983,1.017297,3.238022,0.768611
std,14.101457,0.486057,50.553676,9.671438,2.694694,1.374066,0.892066,1.051290,1.263514,0.706116,1.140061
min,1.000000,0.000000,16.010000,-16.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,0.000000,55.410000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,10.000000,0.000000,68.300000,9.000000,4.000000,2.000000,0.000000,0.000000,1.000000,3.000000,0.000000
75%,18.000000,1.000000,78.525000,16.000000,6.000000,3.000000,1.000000,1.000000,2.000000,4.000000,1.000000
max,202.000000,1.000000,306.880000,47.000000,22.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [5]:
rowsWithNaN = sum(mimicDF.isnull().any(axis=1))
print(f"{mimicDF.shape[0]} rows in df, {rowsWithNaN} containing NaN values")

5262 rows in df, 4778 containing NaN values


In [6]:
for col in mimicDF:
    mimicDF[col] = mimicDF[col].fillna(mimicDF[col].mode()[0])

In [7]:
mimicDF['respiration'].mode()[0]

3.0

In [8]:
mimicDF

,los,gender,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
0,9,1,74.19,7,7,0.0,1.0,0.0,0.0,3.0,3.0
1,2,0,75.00,4,2,3.0,0.0,0.0,1.0,3.0,1.0
2,16,0,51.92,4,2,3.0,0.0,0.0,0.0,3.0,2.0
3,12,0,51.73,-13,4,2.0,2.0,0.0,0.0,3.0,0.0
4,7,0,62.07,0,3,0.0,2.0,0.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5257,120,1,36.61,5,10,3.0,3.0,4.0,0.0,3.0,0.0
5258,13,0,53.90,12,4,3.0,0.0,0.0,0.0,3.0,1.0
5259,4,0,58.93,18,5,3.0,1.0,0.0,4.0,3.0,0.0
5260,16,0,68.98,-4,3,1.0,2.0,0.0,0.0,3.0,0.0


In [9]:
for i in range(len(mimicDF['sofa'])):
    print(mimicDF['sofa'][i])

7
2
2
4
3
7
4
14
8
9
6
5
2
4
2
9
6
3
7
2
3
5
4
6
4
13
2
3
5
4
5
7
4
2
2
2
3
6
2
10
7
4
7
6
7
5
2
10
9
2
6
4
6
4
2
4
3
2
4
4
3
8
3
7
9
4
7
2
12
7
5
4
7
4
10
12
7
3
3
6
5
10
7
3
4
2
6
2
5
6
2
2
7
3
2
10
2
5
9
5
2
8
5
10
3
2
2
3
4
6
2
3
5
9
5
3
2
2
2
4
2
4
2
6
3
7
3
3
5
2
2
3
16
5
2
9
5
4
3
7
11
3
3
4
2
8
5
5
2
4
7
10
8
14
3
9
5
4
5
3
3
3
4
7
4
4
2
7
3
4
12
2
2
2
4
2
7
4
5
5
4
3
3
9
3
3
6
5
4
4
3
5
5
3
2
8
11
8
2
2
7
2
3
3
4
5
4
8
3
6
6
5
2
5
2
7
3
8
11
2
3
9
7
5
4
7
3
2
5
6
2
9
8
2
7
6
3
11
10
3
5
5
8
6
2
2
2
5
6
3
11
3
3
9
6
2
7
8
4
4
2
5
7
3
3
2
3
6
5
2
7
5
3
2
4
2
2
4
4
4
3
4
3
9
6
6
2
9
4
7
4
3
2
9
3
2
4
5
4
3
2
2
8
3
14
11
5
9
18
5
2
2
3
2
4
3
4
3
4
4
2
3
3
11
2
3
3
2
5
2
8
5
2
9
2
2
9
4
5
4
4
3
2
10
3
3
5
4
5
3
4
3
11
4
3
6
9
4
5
7
5
2
3
4
8
7
2
2
3
3
4
3
10
7
4
9
2
11
5
4
3
14
7
12
3
6
3
8
7
2
3
5
8
4
6
4
3
6
8
7
2
3
5
4
6
8
3
4
12
18
2
5
3
12
9
6
6
6
5
4
2
5
4
6
2
2
4
5
2
5
9
2
4
4
5
4
6
14
3
2
4
4
2
5
2
8
4
3
4
7
10
2
4
3
3
9
11
3
4
7
3
7
5
3
8
3
9
3
5
4
3
4
2
4
3
3
4
2
6
5
4
8


In [10]:
mimicDF.describe()

,los,gender,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
count,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.00000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000
mean,14.668757,0.382554,75.049002,9.801026,4.794185,2.36279,0.670468,0.298556,1.017104,3.058533,0.761307
std,14.101457,0.486057,50.553676,9.671438,2.694694,1.19465,0.890785,0.769595,1.263472,0.364759,1.137079
min,1.000000,0.000000,16.010000,-16.000000,2.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,0.000000,55.410000,2.000000,3.000000,2.00000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,10.000000,0.000000,68.300000,9.000000,4.000000,3.00000,0.000000,0.000000,1.000000,3.000000,0.000000
75%,18.000000,1.000000,78.525000,16.000000,6.000000,3.00000,1.000000,0.000000,2.000000,3.000000,1.000000
max,202.000000,1.000000,306.880000,47.000000,22.000000,4.00000,4.000000,4.000000,4.000000,4.000000,4.000000


In [11]:
rowsWithNaN = sum(mimicDF.isnull().any(axis=1))
print(f"{mimicDF.shape[0]} rows in df, {rowsWithNaN} containing NaN values")

5262 rows in df, 0 containing NaN values


#### Replaced NaN values with average of respective columns

In [12]:
dataNeedingEncodingDF = mimicDF[['los', 'age', 'comorbidity', 'sofa']]

In [13]:
bins = 5

est = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='quantile')
categorisedDF = pd.DataFrame(data=est.fit_transform(dataNeedingEncodingDF), columns=dataNeedingEncodingDF.columns)

c:\Users\benma\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '


In [14]:
categorisedDF.head()

,los,age,comorbidity,sofa
0,2.0,3.0,2.0,3.0
1,0.0,3.0,1.0,0.0
2,3.0,0.0,1.0,0.0
3,2.0,0.0,0.0,1.0
4,1.0,1.0,1.0,0.0


In [15]:
pd.set_option('display.max_rows', 1000)

In [16]:
for i in range(len(categorisedDF['sofa'])):
    print(categorisedDF['sofa'][i])

3.0
0.0
0.0
1.0
0.0
3.0
1.0
3.0
3.0
3.0
2.0
2.0
0.0
1.0
0.0
3.0
2.0
0.0
3.0
0.0
0.0
2.0
1.0
2.0
1.0
3.0
0.0
0.0
2.0
1.0
2.0
3.0
1.0
0.0
0.0
0.0
0.0
2.0
0.0
3.0
3.0
1.0
3.0
2.0
3.0
2.0
0.0
3.0
3.0
0.0
2.0
1.0
2.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
0.0
3.0
0.0
3.0
3.0
1.0
3.0
0.0
3.0
3.0
2.0
1.0
3.0
1.0
3.0
3.0
3.0
0.0
0.0
2.0
2.0
3.0
3.0
0.0
1.0
0.0
2.0
0.0
2.0
2.0
0.0
0.0
3.0
0.0
0.0
3.0
0.0
2.0
3.0
2.0
0.0
3.0
2.0
3.0
0.0
0.0
0.0
0.0
1.0
2.0
0.0
0.0
2.0
3.0
2.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
2.0
0.0
3.0
0.0
0.0
2.0
0.0
0.0
0.0
3.0
2.0
0.0
3.0
2.0
1.0
0.0
3.0
3.0
0.0
0.0
1.0
0.0
3.0
2.0
2.0
0.0
1.0
3.0
3.0
3.0
3.0
0.0
3.0
2.0
1.0
2.0
0.0
0.0
0.0
1.0
3.0
1.0
1.0
0.0
3.0
0.0
1.0
3.0
0.0
0.0
0.0
1.0
0.0
3.0
1.0
2.0
2.0
1.0
0.0
0.0
3.0
0.0
0.0
2.0
2.0
1.0
1.0
0.0
2.0
2.0
0.0
0.0
3.0
3.0
3.0
0.0
0.0
3.0
0.0
0.0
0.0
1.0
2.0
1.0
3.0
0.0
2.0
2.0
2.0
0.0
2.0
0.0
3.0
0.0
3.0
3.0
0.0
0.0
3.0
3.0
2.0
1.0
3.0
0.0
0.0
2.0
2.0
0.0
3.0
3.0
0.0
3.0
2.0
0.0
3.0
3.0
0.0
2.0
2.0
3.0
2.0
0.0
0.0
0.0
2.0
2.0
0.0


In [17]:
categorisedDF.describe()

,los,age,comorbidity,sofa
count,5262.000000,5262.000000,5262.000000,5262.000000
mean,2.093691,2.000380,2.089700,1.258267
std,1.354262,1.414751,1.377561,1.200964
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,0.000000
50%,2.000000,2.000000,2.000000,1.000000
75%,3.000000,3.000000,3.000000,2.000000
max,4.000000,4.000000,4.000000,3.000000


In [18]:
categories = {0: 'very_low', 1: 'low', 2: 'medium', 3: 'high', 4: 'very_high'}

encodedDF = categorisedDF.copy()

encodedDF[['respiration','coagulation','liver','renal','cardiovascular','cns']] = mimicDF[['respiration','coagulation','liver','renal','cardiovascular','cns']]

encodedDF

,los,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
0,2.0,3.0,2.0,3.0,0.0,1.0,0.0,0.0,3.0,3.0
1,0.0,3.0,1.0,0.0,3.0,0.0,0.0,1.0,3.0,1.0
2,3.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,3.0,2.0
3,2.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,3.0,0.0
4,1.0,1.0,1.0,0.0,0.0,2.0,0.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...
5257,4.0,0.0,1.0,3.0,3.0,3.0,4.0,0.0,3.0,0.0
5258,3.0,1.0,3.0,1.0,3.0,0.0,0.0,0.0,3.0,1.0
5259,0.0,1.0,4.0,2.0,3.0,1.0,0.0,4.0,3.0,0.0
5260,3.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,3.0,0.0


In [19]:
for col in encodedDF.columns:
    encodedDF[col] = encodedDF[col].map(categories)

encodedDF['gender'] = mimicDF['gender'].map({0:'male', 1:'female'})

encodedDF = pd.get_dummies(encodedDF)

encodedDF.head()

,los_high,los_low,los_medium,los_very_high,los_very_low,age_high,age_low,age_medium,age_very_high,age_very_low,...,cardiovascular_medium,cardiovascular_very_high,cardiovascular_very_low,cns_high,cns_low,cns_medium,cns_very_high,cns_very_low,gender_female,gender_male
0,0,0,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
3,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
4,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [20]:
encodedDF.describe()

,los_high,los_low,los_medium,los_very_high,los_very_low,age_high,age_low,age_medium,age_very_high,age_very_low,...,cardiovascular_medium,cardiovascular_very_high,cardiovascular_very_low,cns_high,cns_low,cns_medium,cns_very_high,cns_very_low,gender_female,gender_male
count,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,...,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000
mean,0.213037,0.255796,0.187761,0.205815,0.137590,0.200304,0.199924,0.199544,0.200114,0.200114,...,0.037818,0.096921,0.000190,0.075447,0.197453,0.088559,0.040099,0.598442,0.382554,0.617446
std,0.409492,0.436349,0.390559,0.404335,0.344502,0.400266,0.399981,0.399695,0.400124,0.400124,...,0.190775,0.295879,0.013786,0.264136,0.398115,0.284134,0.196210,0.490260,0.486057,0.486057
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### TODO Do we want to standardise the data before categorising?

If another point (eg an outlier) is added to the dataset, then this could push the other points away from their original categorisation. Need outlier detection? 

In [21]:
print(encodedDF.columns)

Index(['los_high', 'los_low', 'los_medium', 'los_very_high', 'los_very_low',
       'age_high', 'age_low', 'age_medium', 'age_very_high', 'age_very_low',
       'comorbidity_high', 'comorbidity_low', 'comorbidity_medium',
       'comorbidity_very_high', 'comorbidity_very_low', 'sofa_high',
       'sofa_low', 'sofa_medium', 'sofa_very_low', 'respiration_high',
       'respiration_low', 'respiration_medium', 'respiration_very_high',
       'respiration_very_low', 'coagulation_high', 'coagulation_low',
       'coagulation_medium', 'coagulation_very_high', 'coagulation_very_low',
       'liver_high', 'liver_low', 'liver_medium', 'liver_very_high',
       'liver_very_low', 'renal_high', 'renal_low', 'renal_medium',
       'renal_very_high', 'renal_very_low', 'cardiovascular_high',
       'cardiovascular_medium', 'cardiovascular_very_high',
       'cardiovascular_very_low', 'cns_high', 'cns_low', 'cns_medium',
       'cns_very_high', 'cns_very_low', 'gender_female', 'gender_male'],
      dty

In [22]:
targetDF.describe()

,deathperiod
count,5262.000000
mean,0.896047
std,7.338713
min,-29.000000
25%,-1.000000
50%,-1.000000
75%,-1.000000
max,107.000000


In [23]:
bins = 5

est = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='uniform')
targetCategorisedDF = pd.DataFrame(data=est.fit_transform(targetDF))

targetCategorisedDF = targetCategorisedDF.iloc[:,0].map({1.0: 'very_low', 2.0: 'low', 3.0: 'medium', 4.0: 'high', 5.0: 'very_high'})

targetCategorisedDF = pd.get_dummies(targetCategorisedDF)

targetCategorisedDF

,high,low,medium,very_low
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
5257,0,0,0,1
5258,0,0,0,1
5259,0,0,0,1
5260,0,0,0,1


In [24]:
scorer = Scorer(encodedDF, targetCategorisedDF)

scorer.score()

tensor([[0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        ...,
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1]])
tensor(1.3902, grad_fn=<AddBackward0>)
tensor(1.3639, grad_fn=<AddBackward0>)
tensor(1.3398, grad_fn=<AddBackward0>)
tensor(1.3158, grad_fn=<AddBackward0>)
tensor(1.2883, grad_fn=<AddBackward0>)
tensor(1.2635, grad_fn=<AddBackward0>)
tensor(1.2331, grad_fn=<AddBackward0>)
tensor(1.2060, grad_fn=<AddBackward0>)
tensor(1.1742, grad_fn=<AddBackward0>)
tensor(1.1402, grad_fn=<AddBackward0>)
tensor(1.1045, grad_fn=<AddBackward0>)
tensor(1.0670, grad_fn=<AddBackward0>)
tensor(1.0260, grad_fn=<AddBackward0>)
tensor(0.9779, grad_fn=<AddBackward0>)
tensor(0.9340, grad_fn=<AddBackward0>)
tensor(0.8867, grad_fn=<AddBackward0>)
tensor(0.8382, grad_fn=<AddBackward0>)
tensor(0.7903, grad_fn=<AddBackward0>)
tensor(0.7404, grad_fn=<AddBackward0>)
tensor(0.6909, grad_fn=<AddBackward0>)
tensor(0.6364, grad_fn=<AddBackward0>)
tensor(0.5841, grad_fn=<AddBackward